In [2]:
from dataset import SketchDataset

dataset = SketchDataset(root="data/")

In [8]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from loss import reconstruction_loss

dset = TensorDataset(dataset.nodes, dataset.edges)
loader = DataLoader(dataset = dset, batch_size = 1024)

for nodes, edges in tqdm(loader):
    nodes = nodes.to('cuda')
    edges = edges.to('cuda')
    
    # node_params_mask = node_params_mask.to('cuda')

  0%|          | 0/3889 [00:19<?, ?it/s]


In [7]:
import importlib
import sys
importlib.reload(sys.modules['loss'])
from loss import reconstruction_loss

In [ ]:
from tqdm import trange
for i in trange(len(dataset)):
    nodes, edges = dataset[i]
    nodes = nodes.to('cuda')
    edges = edges.to('cuda')
    # node_params_mask = node_params_mask.to('cuda')